In [ ]:
from google.colab import files

In [ ]:
files.upload()


Saving Pepper.zip to Pepper.zip


In [ ]:
! mkdir pepper

In [ ]:
! unzip /content/Pepper.zip -d pepper

Archive:  /content/Pepper.zip
   creating: pepper/Pepper/
   creating: pepper/Pepper/healthy/
  inflating: pepper/Pepper/healthy/IMG20210206132159_01.jpg  
  inflating: pepper/Pepper/healthy/IMG20210206131948.jpg  
  inflating: pepper/Pepper/healthy/IMG20210206132435.jpg  
  inflating: pepper/Pepper/healthy/IMG20210206132452.jpg  
  inflating: pepper/Pepper/healthy/IMG20210206131947.jpg  
  inflating: pepper/Pepper/healthy/IMG20210206132331_01.jpg  
  inflating: pepper/Pepper/healthy/IMG20210206131853_01.jpg  
  inflating: pepper/Pepper/healthy/IMG20210206131909_01.jpg  
  inflating: pepper/Pepper/healthy/IMG20210206132634.jpg  
  inflating: pepper/Pepper/healthy/IMG20210206132955_01.jpg  
  inflating: pepper/Pepper/healthy/IMG20210206131934.jpg  
  inflating: pepper/Pepper/healthy/IMG20210206132259.jpg  
  inflating: pepper/Pepper/healthy/IMG20210206132158.jpg  
  inflating: pepper/Pepper/healthy/IMG20210206132200.jpg  
  inflating: pepper/Pepper/healthy/IMG20210206131953.jpg  
  infl

In [ ]:
!pip install tensorflow-gpu

     |████████████████████████████████| 394.3MB 33kB/s 


In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
#import matplotlib.pyplot as plt

In [ ]:
import shutil
shutil.rmtree('/content/output')
! mkdir output

In [ ]:
pip install split-folders

In [ ]:
import splitfolders
#split-folders.ratio('/content/pepper/Pepper', output="output", seed=1337, ratio=(.8, 0.1,0.1)) 
splitfolders.ratio("/content/pepper/Pepper", output="/content/output", seed=1337, ratio=(.8, .1, .1), group_prefix=None) # default values

Copying files: 311 files [00:00, 6676.16 files/s]


In [ ]:

# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = '/content/output/train'
valid_path = '/content/output/val'

In [ ]:
# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights

inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [ ]:

# don't train existing weights
for layer in inception.layers:
    layer.trainable = False

In [ ]:
# useful for getting number of output classes
folders = glob('/content/output/train/*')

In [ ]:
folders

['/content/output/train/diseased', '/content/output/train/healthy']

In [ ]:
# our layers - you can add more if you want
x = Flatten()(inception.output)

#https://stackoverflow.com/questions/54797065/resnet-for-binary-classification-just-2-values-of-cross-validation-accuracy

In [ ]:
prediction = Dense(1, activation='sigmoid')(x)


# create a model object
model = Model(inputs=inception.input, outputs=prediction)

In [ ]:
# view the structure of the model
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_188 (Conv2D)             (None, 111, 111, 32) 864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_188 (BatchN (None, 111, 111, 32) 96          conv2d_188[0][0]                 
__________________________________________________________________________________________________
activation_188 (Activation)     (None, 111, 111, 32) 0           batch_normalization_188[0][0]    
____________________________________________________________________________________________

In [ ]:

# tell the model what cost and optimization method to use
model.compile(
  loss='binary_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/output/train/',
                                                 target_size = (224, 224),
                                                 batch_size = 8,
                                                 class_mode = 'binary')

Found 248 images belonging to 2 classes.


In [ ]:
labels = (training_set.class_indices)
print(labels)

{'diseased': 0, 'healthy': 1}


In [ ]:
test_set = test_datagen.flow_from_directory('/content/output/val',
                                            target_size = (224, 224),
                                            batch_size = 8,
                                            class_mode = 'binary')

Found 30 images belonging to 2 classes.


In [ ]:
labels = (test_set.class_indices)
print(labels)


{'diseased': 0, 'healthy': 1}


In [ ]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=3,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/3
31/31 [==============================] - 29s 841ms/step - loss: 1.9042 - accuracy: 0.5945 - val_loss: 0.7890 - val_accuracy: 0.8667
Epoch 2/3
31/31 [==============================] - 25s 819ms/step - loss: 0.7547 - accuracy: 0.8525 - val_loss: 0.3964 - val_accuracy: 0.8667
Epoch 3/3
31/31 [==============================] - 26s 827ms/step - loss: 0.1411 - accuracy: 0.9642 - val_loss: 0.9835 - val_accuracy: 0.8000


In [ ]:
import os
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
disease = ['diseased', 'healthy']
%matplotlib inline
from matplotlib import pyplot as plt
dir_path="/content/output/test/diseased/"
k=0
count=0
for i in os.listdir(dir_path):
  k=k+1
  img = image.load_img(dir_path+i, target_size=(224, 224))
  # Preprocessing the image
  x = image.img_to_array(img)
  # x = np.true_divide(x, 255)
  ## Scaling
  x=x/255
  x = np.expand_dims(x, axis=0)
  preds = model.predict(x)
  if disease[np.argmax(preds)] =='diseased':
    count=count+1
  #print(disease[np.argmax(preds)])
print(k)
print(count)
print(count/k)

13
13
1.0
